In [20]:
import torch
import numpy as np
import torch
import torch.nn as nn

from dialectic import DialecticImitation
from envs import make_env
import importlib
from common.seed import set_seed

from omegaconf import OmegaConf
cfg = OmegaConf.load('dialectic_config.yaml')
set_seed(cfg.seed)


In [25]:
from copy import deepcopy
import warnings

import gym

from envs.wrappers.multitask import MultitaskWrapper
from envs.wrappers.pixels import PixelWrapper
from envs.wrappers.tensor import TensorWrapper

def missing_dependencies(task):
	raise ValueError(f'Missing dependencies for task {task}; install dependencies to use this environment.')

try:
	from envs.dmcontrol import make_env as make_dm_control_env
except:
	make_dm_control_env = missing_dependencies
try:
	from envs.maniskill import make_env as make_maniskill_env
except:
	make_maniskill_env = missing_dependencies
try:
	from envs.metaworld import make_env as make_metaworld_env
except:
	make_metaworld_env = missing_dependencies
try:
	from envs.myosuite import make_env as make_myosuite_env
except:
	make_myosuite_env = missing_dependencies


warnings.filterwarnings('ignore', category=DeprecationWarning)

def make_env(cfg):
	"""
	Make an environment for TD-MPC2 experiments.
	"""
	gym.logger.set_level(40)
	# if cfg.multitask:
	if False:
		env = make_multitask_env(cfg)
	else:
		env = None
		for fn in [make_dm_control_env, make_maniskill_env, make_metaworld_env, make_myosuite_env]:
			try:
				env = fn(cfg)
			except ValueError:
				pass
		if env is None:
			raise ValueError(f'Failed to make environment "{cfg.task}": please verify that dependencies are installed and that the task exists.')
		env = TensorWrapper(env)
	if cfg.get('obs', 'state') == 'rgb':
		env = PixelWrapper(cfg, env)
	try: # Dict
		cfg.obs_shape = {k: v.shape for k, v in env.observation_space.spaces.items()}
	except: # Box
		cfg.obs_shape = {cfg.get('obs', 'state'): env.observation_space.shape}
	cfg.action_dim = env.action_space.shape[0]
	cfg.episode_length = env.max_episode_steps
	cfg.seed_steps = max(1000, 5*cfg.episode_length)
	return env


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 42)

In [21]:
model = DialecticImitation(cfg)

In [22]:
evn = make_env(cfg)

MissingMandatoryValue: Missing mandatory value: multitask
    full_key: multitask
    object_type=dict

In [14]:
model

In [9]:
a = torch.rand(2,5)
b = torch.rand(2,5)

print(a)
print(b)

tensor([[0.8388, 0.0245, 0.9674, 0.1123, 0.6872],
        [0.5021, 0.7984, 0.1066, 0.3427, 0.8748]])
tensor([[0.8409, 0.4682, 0.5965, 0.0198, 0.2972],
        [0.7504, 0.8262, 0.0602, 0.1317, 0.4913]])


In [11]:
torch.cat((a,b), dim=1)


tensor([[0.8388, 0.0245, 0.9674, 0.1123, 0.6872, 0.8409, 0.4682, 0.5965, 0.0198,
         0.2972],
        [0.5021, 0.7984, 0.1066, 0.3427, 0.8748, 0.7504, 0.8262, 0.0602, 0.1317,
         0.4913]])